## Creation of a corpus based on the body of our CSV.

In [2]:
#import libraries 
import numpy as np
import pandas as pd
from pathlib import Path


In [4]:
df = pd.read_csv(r"data\v2.csv")

In [5]:
df.shape

(128103, 8)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128103 entries, 0 to 128102
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  128103 non-null  int64 
 1   id_mail     128103 non-null  object
 2   date        128103 non-null  object
 3   from        128103 non-null  object
 4   to          125357 non-null  object
 5   subject     123580 non-null  object
 6   body        128103 non-null  object
 7   x_origin    128103 non-null  object
dtypes: int64(1), object(7)
memory usage: 7.8+ MB


In [10]:
df["body"].head(10)

0    ---------------------- Forwarded by Phillip K ...
1    ---------------------- Forwarded by Phillip K ...
2    Dave, \n\n Here are the names of the west desk...
3             Paula,\n\n 35 million is fine\n\nPhillip
4    ---------------------- Forwarded by Phillip K ...
5    ---------------------- Forwarded by Phillip K ...
6    ---------------------- Forwarded by Phillip K ...
7    Brenda,\n\nPlease use the second check as the ...
8    I think Fletch has a good CPA.  I am still doi...
9    Brenda,\n\n Please use the second check as my ...
Name: body, dtype: object

In [25]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alebe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alebe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [22]:
from nltk.corpus import stopwords
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer
import re

In [31]:
def clean(text):
    stop = set(stopwords.words('english'))
    stop.update(("to","cc","subject","http","from","sent","aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    porter= PorterStemmer()
    
    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    stem = " ".join(porter.stem(token) for token in normalized.split())
    
    return stem

In [26]:
body_cleaned = []
for body in df["body"]:
    body_cleaned.append(clean(body).split())


In [27]:
body_cleaned

[['forwarded',
  'phillip',
  'k',
  'allen',
  'hou',
  'ect',
  'pm',
  'richard',
  'burchfield',
  'phillip',
  'k',
  'allen',
  'hou',
  'ect',
  'ect',
  'beth',
  'perlman',
  'hou',
  'ect',
  'ect',
  'consolidated',
  'position',
  'issue',
  'list',
  'phillip',
  'issue',
  'list',
  'go',
  'forward',
  'documenting',
  'requirement',
  'consolidated',
  'physical',
  'financial',
  'position',
  'transport',
  'trade',
  'capture',
  'need',
  'focus',
  'first',
  'bullet',
  'allan',
  'list',
  'need',
  'single',
  'set',
  'requirement',
  'although',
  'meeting',
  'keith',
  'wednesday',
  'informative',
  'solution',
  'creating',
  'infinitely',
  'dynamic',
  'consolidated',
  'position',
  'screen',
  'extremely',
  'difficult',
  'time',
  'consuming',
  'throughout',
  'meeting',
  'wednesday',
  'keith',
  'alluded',
  'inability',
  'get',
  'consensus',
  'amongst',
  'trader',
  'presentation',
  'consolidated',
  'position',
  'solution',
  'make',
  't

In [28]:
len(body_cleaned)

128103

In [29]:
body_cleaned[0]

['forwarded',
 'phillip',
 'k',
 'allen',
 'hou',
 'ect',
 'pm',
 'richard',
 'burchfield',
 'phillip',
 'k',
 'allen',
 'hou',
 'ect',
 'ect',
 'beth',
 'perlman',
 'hou',
 'ect',
 'ect',
 'consolidated',
 'position',
 'issue',
 'list',
 'phillip',
 'issue',
 'list',
 'go',
 'forward',
 'documenting',
 'requirement',
 'consolidated',
 'physical',
 'financial',
 'position',
 'transport',
 'trade',
 'capture',
 'need',
 'focus',
 'first',
 'bullet',
 'allan',
 'list',
 'need',
 'single',
 'set',
 'requirement',
 'although',
 'meeting',
 'keith',
 'wednesday',
 'informative',
 'solution',
 'creating',
 'infinitely',
 'dynamic',
 'consolidated',
 'position',
 'screen',
 'extremely',
 'difficult',
 'time',
 'consuming',
 'throughout',
 'meeting',
 'wednesday',
 'keith',
 'alluded',
 'inability',
 'get',
 'consensus',
 'amongst',
 'trader',
 'presentation',
 'consolidated',
 'position',
 'solution',
 'make',
 'trader',
 'arrange',
 'position',
 'screen',
 'liking',
 'much',
 'like',
 'excel

In [30]:
df["body"][0]

"---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/09/2000 \n02:16 PM ---------------------------\n\n\nRichard Burchfield\n10/06/2000 06:59 AM\nTo: Phillip K Allen/HOU/ECT@ECT\ncc: Beth Perlman/HOU/ECT@ECT \nSubject: Consolidated positions: Issues & To Do list\n\nPhillip,\n Below is the issues & to do list as we go forward with documenting the \nrequirements for consolidated physical/financial positions and transport \ntrade capture. What we need to focus on is the first bullet in Allan's list; \nthe need for a single set of requirements. Although the meeting with Keith, \non Wednesday,  was informative the solution of creating a infinitely dynamic \nconsolidated position screen, will be extremely difficult and time \nconsuming.  Throughout the meeting on Wednesday, Keith alluded to the \ninability to get consensus amongst the traders on the presentation of the \nconsolidated position, so the solution was to make it so that a trader can \narrange the position screen to 

In [32]:
#try with tokenization
body_token = []
for body in df["body"]:
    body_token.append(clean(body).split())

In [33]:
body_token[0]

['forward',
 'phillip',
 'k',
 'allen',
 'hou',
 'ect',
 'pm',
 'richard',
 'burchfield',
 'phillip',
 'k',
 'allen',
 'hou',
 'ect',
 'ect',
 'beth',
 'perlman',
 'hou',
 'ect',
 'ect',
 'consolid',
 'posit',
 'issu',
 'list',
 'phillip',
 'issu',
 'list',
 'go',
 'forward',
 'document',
 'requir',
 'consolid',
 'physic',
 'financi',
 'posit',
 'transport',
 'trade',
 'captur',
 'need',
 'focu',
 'first',
 'bullet',
 'allan',
 'list',
 'need',
 'singl',
 'set',
 'requir',
 'although',
 'meet',
 'keith',
 'wednesday',
 'inform',
 'solut',
 'creat',
 'infinit',
 'dynam',
 'consolid',
 'posit',
 'screen',
 'extrem',
 'difficult',
 'time',
 'consum',
 'throughout',
 'meet',
 'wednesday',
 'keith',
 'allud',
 'inabl',
 'get',
 'consensu',
 'amongst',
 'trader',
 'present',
 'consolid',
 'posit',
 'solut',
 'make',
 'trader',
 'arrang',
 'posit',
 'screen',
 'like',
 'much',
 'like',
 'excel',
 'need',
 'happen',
 'monday',
 'effort',
 'design',
 'desir',
 'layout',
 'consolid',
 'posit',
 

In [36]:
for list in body_cleaned:
    for word in list:
        if len(word) <= 3:
            list.remove(word)


In [37]:
body_cleaned[0]

['forwarded',
 'phillip',
 'allen',
 'richard',
 'burchfield',
 'phillip',
 'allen',
 'beth',
 'perlman',
 'consolidated',
 'position',
 'issue',
 'list',
 'phillip',
 'issue',
 'list',
 'forward',
 'documenting',
 'requirement',
 'consolidated',
 'physical',
 'financial',
 'position',
 'transport',
 'trade',
 'capture',
 'need',
 'focus',
 'first',
 'bullet',
 'allan',
 'list',
 'need',
 'single',
 'requirement',
 'although',
 'meeting',
 'keith',
 'wednesday',
 'informative',
 'solution',
 'creating',
 'infinitely',
 'dynamic',
 'consolidated',
 'position',
 'screen',
 'extremely',
 'difficult',
 'time',
 'consuming',
 'throughout',
 'meeting',
 'wednesday',
 'keith',
 'alluded',
 'inability',
 'consensus',
 'amongst',
 'trader',
 'presentation',
 'consolidated',
 'position',
 'solution',
 'make',
 'trader',
 'arrange',
 'position',
 'screen',
 'liking',
 'much',
 'like',
 'excel',
 'need',
 'happen',
 'monday',
 'effort',
 'design',
 'desired',
 'layout',
 'consolidated',
 'position

In [39]:
len(body_cleaned[0])

473